In [36]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

In [37]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [38]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset,  test_labels  = reformat(test_dataset, test_labels)

print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [39]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
        / predictions.shape[0])

In [41]:
# Problem 1 : Add regularize term in the Logistic Regression model
train_subset = 10000
penal = 0.1

graph = tf.Graph()
with graph.as_default():

    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_labels])) # 784(pixel num) * 10(label num)
    biases = tf.Variable(tf.zeros([num_labels]))
  
    logits = tf.matmul(tf_train_dataset, weights) + biases # inner product + summation
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) + 
        penal * tf.nn.l2_loss(weights))
  
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction  = tf.nn.softmax(tf.matmul(tf_test_dataset,  weights) + biases)

In [42]:
num_steps = 801

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        _, l, predictions = session.run([optimizer, loss, train_prediction])
        if (step % 100 == 0):
            print('Loss at step %d: %f' % (step, l))
            print('Training accuracy: %.1f%%' % accuracy(predictions, train_labels[:train_subset, :]))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 320.846985
Training accuracy: 7.2%
Validation accuracy: 8.6%
Loss at step 100: 1.021584
Training accuracy: 80.5%
Validation accuracy: 80.6%
Loss at step 200: 1.009919
Training accuracy: 80.5%
Validation accuracy: 80.5%
Loss at step 300: 1.009328
Training accuracy: 80.6%
Validation accuracy: 80.7%
Loss at step 400: 1.009156
Training accuracy: 80.6%
Validation accuracy: 80.7%
Loss at step 500: 1.009100
Training accuracy: 80.6%
Validation accuracy: 80.7%
Loss at step 600: 1.009081
Training accuracy: 80.6%
Validation accuracy: 80.7%
Loss at step 700: 1.009074
Training accuracy: 80.6%
Validation accuracy: 80.7%
Loss at step 800: 1.009072
Training accuracy: 80.6%
Validation accuracy: 80.7%
Test accuracy: 87.6%


In [11]:
# Problem 2 : Demonstrate overfitting
batch_size = 5

graph = tf.Graph()
with graph.as_default():

    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
  
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction  = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [12]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]

        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 13.931902
Minibatch accuracy: 0.0%
Validation accuracy: 13.8%
Minibatch loss at step 500: 0.000898
Minibatch accuracy: 100.0%
Validation accuracy: 72.8%
Minibatch loss at step 1000: 21.011303
Minibatch accuracy: 40.0%
Validation accuracy: 72.4%
Minibatch loss at step 1500: 8.082113
Minibatch accuracy: 40.0%
Validation accuracy: 68.2%
Minibatch loss at step 2000: 3.312748
Minibatch accuracy: 80.0%
Validation accuracy: 75.0%
Minibatch loss at step 2500: 1.818836
Minibatch accuracy: 40.0%
Validation accuracy: 69.4%
Minibatch loss at step 3000: 2.081526
Minibatch accuracy: 80.0%
Validation accuracy: 71.5%
Test accuracy: 78.1%


In [15]:
# Problem 3 : Introduce Dropout
train_subset = 10000
penal = 0.3
batch_size = 128
keep_prob = 0.5

graph = tf.Graph()
with graph.as_default():

    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    weights_d = tf.nn.dropout(weights, keep_prob=keep_prob)
  
    logits = tf.matmul(tf_train_dataset, weights_d) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction  = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [16]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]

        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 27.257832
Minibatch accuracy: 4.7%
Validation accuracy: 6.7%
Minibatch loss at step 500: 6.349340
Minibatch accuracy: 64.1%
Validation accuracy: 80.7%
Minibatch loss at step 1000: 2.892833
Minibatch accuracy: 72.7%
Validation accuracy: 81.2%
Minibatch loss at step 1500: 2.332426
Minibatch accuracy: 67.2%
Validation accuracy: 81.8%
Minibatch loss at step 2000: 1.915910
Minibatch accuracy: 67.2%
Validation accuracy: 80.6%
Minibatch loss at step 2500: 1.426371
Minibatch accuracy: 68.8%
Validation accuracy: 79.4%
Minibatch loss at step 3000: 0.803453
Minibatch accuracy: 81.2%
Validation accuracy: 81.8%
Test accuracy: 88.6%


In [60]:
# Problem 4 : Try multi-layer model
batch_size = 128
n_input = image_size * image_size
n_hl_1 = 1024
n_hl_2 = 305
n_hl_3 = 75
keep_prob = 0.5

graph = tf.Graph()
with graph.as_default():

    # set placeholder
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, n_input))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # initialization
    w1 = tf.Variable(tf.truncated_normal([n_input, n_hl_1], stddev=0.0517))
    b1 = tf.Variable(tf.zeros([n_hl_1]))
    w2 = tf.Variable(tf.truncated_normal([n_hl_1, n_hl_2], stddev=0.0441))
    b2 = tf.Variable(tf.zeros([n_hl_2]))
    w3 = tf.Variable(tf.truncated_normal([n_hl_2, n_hl_3], stddev=0.0441))
    b3 = tf.Variable(tf.zeros([n_hl_3]))
    w4 = tf.Variable(tf.truncated_normal([n_hl_3, num_labels], stddev=0.0809))
    b4 = tf.Variable(tf.zeros([num_labels]))


    # model
    v1 = tf.nn.relu(tf.matmul(tf.nn.dropout(tf_train_dataset, keep_prob=keep_prob), w1) + b1)
    v2 = tf.nn.relu(tf.matmul(tf.nn.dropout(v1, keep_prob=keep_prob), w2) + b2)
    v3 = tf.nn.relu(tf.matmul(tf.nn.dropout(v2, keep_prob=keep_prob), w3) + b3)
    v4 = tf.nn.relu(tf.matmul(tf.nn.dropout(v3, keep_prob=keep_prob), w4) + b4)
    
#    v1 = tf.nn.dropout(tf.nn.relu(tf.matmul(tf_train_dataset, w1) + b1), keep_prob=keep_prob)
#    v2 = tf.nn.dropout(tf.nn.relu(tf.matmul(v1, w2) + b2), keep_prob=keep_prob)
#    v3 = tf.nn.dropout(tf.nn.relu(tf.matmul(v2, w3) + b3), keep_prob=keep_prob)

    logits = v4

    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) 


#    global_step = tf.Variable(0)  # count the number of steps taken.
#    learning_rate = tf.train.exponential_decay(0.3, global_step, 3500, 0.86, staircase=True)
     
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
#    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)


  
    train_prediction = tf.nn.softmax(logits)
#    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(
#                    tf_valid_dataset, weight1) + bias1), weight2) + bias2)
#    test_prediction  = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(
#                    tf_test_dataset, weight1) + bias1), weight2) + bias2)

In [61]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]

        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
#            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
#    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.321219
Minibatch accuracy: 11.7%
Minibatch loss at step 500: 0.894441
Minibatch accuracy: 72.7%
Minibatch loss at step 1000: 0.579687
Minibatch accuracy: 82.8%
Minibatch loss at step 1500: 0.671782
Minibatch accuracy: 81.2%
Minibatch loss at step 2000: 0.633635
Minibatch accuracy: 81.2%
Minibatch loss at step 2500: 0.764913
Minibatch accuracy: 78.9%
Minibatch loss at step 3000: 0.507025
Minibatch accuracy: 83.6%


In [62]:
# Sample code
batch_size = 128
hidden_layer1_size = 1024
hidden_layer2_size = 305
hidden_lastlayer_size = 75

use_multilayers = True
regularization_meta=0.03

graph = tf.Graph()
with graph.as_default():

    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    keep_prob = tf.placeholder(tf.float32)

    weights_layer1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, hidden_layer1_size], stddev=0.0517))
    biases_layer1 = tf.Variable(tf.zeros([hidden_layer1_size]))

    if use_multilayers:
        weights_layer2 = tf.Variable(
            tf.truncated_normal([hidden_layer1_size, hidden_layer1_size], stddev=0.0441))
        biases_layer2 = tf.Variable(tf.zeros([hidden_layer1_size]))

        weights_layer3 = tf.Variable(
            tf.truncated_normal([hidden_layer1_size, hidden_layer2_size], stddev=0.0441))
        biases_layer3 = tf.Variable(tf.zeros([hidden_layer2_size]))
    
        weights_layer4 = tf.Variable(
            tf.truncated_normal([hidden_layer2_size, hidden_lastlayer_size], stddev=0.0809))
        biases_layer4 = tf.Variable(tf.zeros([hidden_lastlayer_size]))


    weights = tf.Variable(
        tf.truncated_normal([hidden_lastlayer_size if use_multilayers else hidden_layer1_size, 
                             num_labels], stddev=0.1632))
    biases = tf.Variable(tf.zeros([num_labels]))
  
    
    # get the NN models
    def getNN4Layer(dSet, use_dropout):
        input_to_layer1 = tf.matmul(dSet, weights_layer1) + biases_layer1
        hidden_layer1_output = tf.nn.relu(input_to_layer1)
        
        logits_hidden1 = None
        if use_dropout:
            dropout_hidden1 = tf.nn.dropout(hidden_layer1_output, keep_prob)
            logits_hidden1 = tf.matmul(dropout_hidden1, weights_layer2) + biases_layer2
        else:
            logits_hidden1 = tf.matmul(hidden_layer1_output, weights_layer2) + biases_layer2

        hidden_layer2_output = tf.nn.relu(logits_hidden1)
    

        logits_hidden2 = None
        if use_dropout:
            dropout_hidden2 = tf.nn.dropout(hidden_layer2_output, keep_prob)
            logits_hidden2 = tf.matmul(dropout_hidden2, weights_layer3) + biases_layer3
        else:
            logits_hidden2 = tf.matmul(hidden_layer2_output, weights_layer3) + biases_layer3
    
        hidden_layer3_output = tf.nn.relu(logits_hidden2)


        logits_hidden3 = None
        if use_dropout:
            dropout_hidden3 = tf.nn.dropout(hidden_layer3_output, keep_prob)
            logits_hidden3 = tf.matmul(dropout_hidden3, weights_layer4) + biases_layer4
        else:
            logits_hidden3 = tf.matmul(hidden_layer3_output, weights_layer4) + biases_layer4
    
        hidden_layer4_output = tf.nn.relu(logits_hidden3)
    
    
        logits = None
        if use_dropout:
            dropout_hidden4 = tf.nn.dropout(hidden_layer4_output, keep_prob)
            logits = tf.matmul(dropout_hidden4, weights) + biases
        else:
            logits = tf.matmul(hidden_layer4_output, weights) + biases
    
        return logits

    
    # get the NN models
    def getNN1Layer(dSet, use_dropout, w1, b1, w, b):
        input_to_layer1 = tf.matmul(dSet, w1) + b1
        hidden_layer1_output = tf.nn.relu(input_to_layer1)
        
        logits = None
        if use_dropout:
            dropout_hidden1 = tf.nn.dropout(hidden_layer1_output, keep_prob)
            logits = tf.matmul(dropout_hidden1, w) + b
        else:
            logits = tf.matmul(hidden_layer1_output, w) + b
    
        return logits

  
    # Training computation.
    logits = getNN4Layer(tf_train_dataset, True)  
    logits_valid = getNN4Layer(tf_valid_dataset, False)
    logits_test = getNN4Layer(tf_test_dataset, False)
    
  
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    #loss_l2 = loss + (regularization_meta * (tf.nn.l2_loss(weights)))
  
    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.3, global_step, 3500, 0.86, staircase=True)
  
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(logits_valid)
    test_prediction = tf.nn.softmax(logits_test)

In [64]:
num_steps = 2000


with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in xrange(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
    
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob:0.75}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step", step, ":", l)
            print("Minibatch accuracy: %.1f%%" % accuracy(train_prediction.eval(feed_dict={tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob:1.0}), batch_labels))
#            print("Minibatch accuracy: %.1f%%" % accuracy(train_prediction(feed_dict={tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob:1.0}), batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(feed_dict={keep_prob:1.0}), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(feed_dict={keep_prob:1.0}), test_labels))

Initialized
Minibatch loss at step 0 : 2.55946
Minibatch accuracy: 32.0%
Validation accuracy: 20.5%
Minibatch loss at step 500 : 0.639841
Minibatch accuracy: 83.6%


KeyboardInterrupt: 